https://colab.research.google.com/github/aaudevart/devoxx2021-TF/blob/main/Demo_CNN/CNN_MNIST.ipynb

# Démo CNN - Classification d'images MNIST avec *tf.keras*

### Objectif :
- Mise en place d'un CNN

Lien vers la documentation de [TensorFlow](https://www.tensorflow.org/api_docs).


#◢  Import des librairies

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

#◢  Chargement du jeu de données MNIST


In [ ]:
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

X_train_full = X_train_full.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

#◢  Pré-Processing
Normalisation des images







In [ ]:
def normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    a = 0.1
    b = 0.9
    image_data_min = image_data.min()
    image_data_max = image_data.max()
    return a + ((image_data - image_data_min)*(b-a))/(image_data_max - image_data_min)

train_features = normalize_grayscale(X_train)
val_features = normalize_grayscale(X_valid)
test_features = normalize_grayscale(X_test)


#◢  Construction d'un modèle 

In [ ]:
model = keras.models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

#◢  Compilation du modèle

Après avoir créé le modèle, nous devons appeler la méthode `compile()`afin de spécifier la fonction de cout (`loss` function) et l'`optimizer` à utiliser. 

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#◢  Entrainement du modèle


In [ ]:
history = model.fit(train_features, y_train, epochs=5,
                    validation_data=(val_features, y_valid))

Regardons les valeurs de la fonction de cout et de l'accuracy sur les 2 jeux de données (train et validation)

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()

In [ ]:
plot_learning_curves(history)

#◢  Evaluation du modèle

In [ ]:
model.evaluate(test_features, y_test)

#◢  Prédiction

In [ ]:
y_proba = model.predict(test_features)
y_pred = y_proba.argmax(axis=1)

#◢  Visualisation des prédictions 

Méthodes utilitaires pour afficher une image et un bar chart représentant la probabilité des prédictions pour chaque chiffre de 0 à 9 (les prédictions en bleu sont les prédictions correctes et en rouge celles incorrectes)

In [ ]:
def plot_prediction_image(predictions_array, true_label, img):

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    color = "blue" if predicted_label == true_label else "red"

    plt.xlabel("Chiffre prédit {} {:2.0f}% ({})".format(predicted_label,
                                                        np.max(predictions_array) *100,
                                                        true_label),
                                                        color=color)
    
def plot_prediction_bar_chart(predictions_array, true_label):
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    plt.grid(False)
    plt.xticks(range(10))
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

Affichage de l'image à prédire et de la probabilité des prédictions pour chaque chiffre de 0 à 9 pour la première image

In [ ]:
i = 1
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_prediction_image(y_proba[i], y_test[i], np.reshape(X_test[i], [28,28]))
plt.subplot(1,2,2)
plot_prediction_bar_chart(y_proba[i], y_test[i])
plt.show()

In [ ]:
y_proba[i]

## Affichons les 20 premiers chiffres et visualisons les prédictions


In [ ]:
num_rows = 8
num_cols = 5
plt.figure(figsize=(2*2*num_cols, 2*num_rows))

for row in range(num_rows):
    for col in range(num_cols):
        index = num_cols * row + col
        plt.subplot(num_rows, 2*num_cols, 2*index+1)
        plot_prediction_image(y_proba[index], y_test[index], np.reshape(X_test[i], [28,28]))
        plt.subplot(num_rows, 2*num_cols, 2*index+2)
        plot_prediction_bar_chart(y_proba[index], y_test[index])
plt.tight_layout()
plt.show()



## Focus sur les images mal prédites

In [ ]:
y_proba_false = y_proba[y_pred != y_test]
y_test_false = y_test[y_pred != y_test]
X_test_false = X_test[y_pred != y_test]

In [ ]:
num_rows = 8
num_cols = 5
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for row in range(num_rows):
    for col in range(num_cols):
        index = num_cols * row + col
        plt.subplot(num_rows, 2*num_cols, 2*index+1)
        plot_prediction_image(y_proba_false[index], y_test_false[index], np.reshape(X_test[index], [28,28]))
        plt.subplot(num_rows, 2*num_cols, 2*index+2)
        plot_prediction_bar_chart(y_proba_false[index], y_test_false[index])
plt.tight_layout()
plt.show()